In [5]:
from logic import *

class ALU():
    def __init__(self, name, zeroFlag = 1, overflowFlag = 0, negativeFlag = 0):
        self.name = name
        self.zeroFlag = zeroFlag
        self.overflowFlag  = overflowFlag
        self.negativeFlag= negativeFlag
        
    @staticmethod
    def toString(byte):
        s = ""
        for bit in byte:
            x = str(int(bit))
            s += x
        return s

    def isZero(self, byte):
        Out = Not("Out")
        def isSimpleZero(byte):
            if len(byte) == 2:
                F0 = Or("F0")
                F0.A.set(bit(byte, 0))
                F0.B.set(bit(byte, 1))
                return F0.C.value
            else:
                F1 = Or("F1")
                left = isSimpleZero(byte[0:int(len(byte)/2)])
                right = isSimpleZero(byte[int(len(byte)/2):int(len(byte))])
                F1.A.set(left)
                F1.B.set(right)
                # print(left, right)
                return (F1.C.value)
        Out.A.set(isSimpleZero(byte))
        Out.evaluate()
        flag = str(int(Out.B.value))
        self.zeroFlag = flag
        if(flag == "1"):
            print("Is Zero: ", self.zeroFlag)
        return flag
    
    def isNegative(self, byte):
        F0 = And("F0")
        F0.A.set(bit(byte, 0))
        F0.B.set(True)
        flag = str(int(F0.C.value))
        self.negativeFlag = flag
        if(flag == "1"):
            print("Is Negative: ", self.negativeFlag)
        return(flag)

    def addCarry(self, a, b):  
        F0 = FullAdder("F0")
        F1 = FullAdder("F1")
        F0.Cout.connect(F1.Cin)
        F2 = FullAdder("F2")
        F1.Cout.connect(F2.Cin)
        F3 = FullAdder("F3")
        F2.Cout.connect(F3.Cin)
        F4 = FullAdder("F4")
        F3.Cout.connect(F4.Cin)
        F5 = FullAdder("F5")
        F4.Cout.connect(F5.Cin)
        F6 = FullAdder("F6")
        F5.Cout.connect(F6.Cin)
        F7 = FullAdder("F7")
        F6.Cout.connect(F7.Cin)
            
        F0.Cin.set(0)
        F0.A.set(bit(a, 7))
        F0.B.set(bit(b, 7)) 
        F1.A.set(bit(a, 6))
        F1.B.set(bit(b, 6))
        F2.A.set(bit(a, 5))
        F2.B.set(bit(b, 5))
        F3.A.set(bit(a, 4))
        F3.B.set(bit(b, 4))
        F4.A.set(bit(a, 3))
        F4.B.set(bit(b, 3))
        F5.A.set(bit(a, 2))
        F5.B.set(bit(b, 2))
        F6.A.set(bit(a, 1))
        F6.B.set(bit(b, 1))
        F7.A.set(bit(a, 0))
        F7.B.set(bit(b, 0))
        
        flag = str(int(F7.Cout.value))
        self.overflowFlag = flag
        if (flag == "1"):
            print("Is Overflow: " , self.overflowFlag )
        array = [F7.S.value,F6.S.value,F5.S.value,
                                    F4.S.value,F3.S.value,F2.S.value,F1.S.value,F0.S.value]
        byte = self.toString(array)
        self.isNegative(byte)
        self.isZero(byte)
        return byte
    
    def increment(self, a):
        return self.toString(self.addCarry(a, "00000001"))

    def decrement(self, a):
        return self.toString(self.addCarry(a, "11111111"))

    def complementOne(self, a):
        byte = []
        for i in a:
            F0 = Not("F0")
            F0.A.set(i == '1')
            F0.evaluate()
            byte.append(F0.B.value)
        return self.toString(byte)

    def negate(self, a):
        complement = self.complementOne(a)
        result = self.increment(complement)
        return self.toString(result)
        
    def passTrough(self, a):
        inverted = self.complementOne(a)
        result = self.complementOne(inverted)
        return self.toString(result)

    def substraction (self, a, b):
        b = self.toString(self.negate(b))
        c = self.addCarry(a, b)
        return self.toString(c)

In [7]:
import unittest

In [18]:
class TestALU(unittest.TestCase):
    test = ALU("test")
    def test01_isZero(self):
        self.assertEqual(self.test.isZero('11000000'),'0'), "Should be False"
    def test02_isZero(self):
        self.assertEqual(self.test.isZero('11111111'), '0'), "Should be False"
    def test03_isZero(self):
        self.assertEqual(self.test.isZero('00000000'),'1'), "Should be True"

    def test04_isNegative(self):
        self.assertEqual(self.test.isNegative('11000000'),'1'), "Should be True"
    def test05_isNegative(self):
        self.assertEqual(self.test.isNegative('11111111'), '1'), "Should be True"
    def test06_isNegative(self):
        self.assertEqual(self.test.isNegative('00000000'),'0'), "Should be False"

    def test07_adderCarry(self):
        self.assertEqual(self.test.addCarry('11000000','10101000'),"01101000"), "Should be 101101000"
    def test08_adderCarry(self):
        self.assertEqual(self.test.addCarry('11000010','00101010'),"11101100"), "Should be 011101100"
    def test09_adderCarry(self):
        self.assertEqual(self.test.addCarry('11000010','00101010'),'11101100'), "Should be 11101100"
    
    def test10_complementOne(self):
        self.assertEqual(self.test.complementOne("11111100"),"00000011"), "Should be 00000011"
    def test11_complementOne(self):
        self.assertEqual(self.test.complementOne("00000000"),"11111111"), "Should be 11111111"
    def test12_complementOne(self):
        self.assertEqual(self.test.complementOne("01010101"),"10101010"), "Should be 10101010"
    
    def test13_negate(self):
        self.assertEqual(self.test.negate("00000001"),"11111111"), "Should be 11111111"
    def test14_negate(self):
        self.assertEqual(self.test.negate("10100101"),"01011011"), "Should be 01011011"
    def test15_negate(self):
        self.assertEqual(self.test.negate("00101100"),"11010100"), "Should be 11010100"

    def test16_substraction(self):
        self.assertEqual(self.test.substraction("00000001", "00000001"),"00000000"), "Should be 00000000"
    def test17_substraction(self):
        self.assertEqual(self.test.substraction('10001001', '10000001'),"00001000"), "Should be 00001000"
    def test18_substraction(self):
        self.assertEqual(self.test.substraction('00001011', '11000001'),"01001010"), "Should be 01001010"
        
    def test19_passTrough(self):
        self.assertEqual(self.test.passTrough("10010001"),"10010001"), "Should be 10010001"
    def test20_passTrough(self):
        self.assertEqual(self.test.passTrough("10100101"),"10100101"), "Should be 10100101"
    def test21_passTrough(self):
        self.assertEqual(self.test.passTrough("00101100"),"00101100"), "Should be 00101100"


unittest.main(argv=[''], verbosity=2, exit=False)

test01_isZero (__main__.TestALU) ... ok
test02_isZero (__main__.TestALU) ... ok
test03_isZero (__main__.TestALU) ... ok
test04_isNegative (__main__.TestALU) ... ok
test05_isNegative (__main__.TestALU) ... ok
test06_isNegative (__main__.TestALU) ... ok
test07_adderCarry (__main__.TestALU) ... ok
test08_adderCarry (__main__.TestALU) ... ok
test09_adderCarry (__main__.TestALU) ... ok
test10_complementOne (__main__.TestALU) ... ok
test11_complementOne (__main__.TestALU) ... ok
test12_complementOne (__main__.TestALU) ... ok
test13_negate (__main__.TestALU) ... ok
test14_negate (__main__.TestALU) ... ok
test15_negate (__main__.TestALU) ... ok
test16_substraction (__main__.TestALU) ... ok
test17_substraction (__main__.TestALU) ... ok
test18_substraction (__main__.TestALU) ... ok
test19_passTrough (__main__.TestALU) ... ok
test20_passTrough (__main__.TestALU) ... ok
test21_passTrough (__main__.TestALU) ... 

Is Zero:  1
Is Negative:  1
Is Negative:  1
Is Overflow:  1
Is Negative:  1
Is Negative:  1
Is Negative:  1
Is Negative:  1
Is Negative:  1
Is Overflow:  1
Is Zero:  1
Is Overflow:  1


ok

----------------------------------------------------------------------
Ran 21 tests in 0.029s

OK
